<a href="https://colab.research.google.com/github/deepakmoud/Signature-Verification/blob/master/Signature_Recognition_Nasnet_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing the Keras libraries and packages
import tensorflow as tf
from tensorflow.python import keras
from keras.layers import Input, Flatten, Dense
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications import Xception
from keras.applications import VGG19
from keras.applications import ResNet152V2
from keras.applications import InceptionV3
from keras.applications import DenseNet121
from keras.applications import NASNetLarge
import warnings 
warnings.filterwarnings('ignore')
img_width, img_height = 331, 331

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Input parameter
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train'
val_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test'
TEST_DIR='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/TEST1/'
filepath = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/vgg16_GDPS_xfer_weights.hdf5'
nb_epochs = 50
SIGNATURE_CLASSES = ['001', '002', '003','004','006','009','012','013','014','015','016','017','018','019','020','021','022','023','024','025','026','027','028','029','030','031','032','033','034','035','036','037','038','039','040','041','042','043','044','045','046','047','048','049','050','051','052','053','054','055','056','057','058','059','060','061','062','063','064','065','066','067','068','069']
print("Input parameters are assigned")

Input parameters are assigned


In [14]:
# image data generation
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train/'
val_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test/'
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=True, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=True,class_mode='categorical')

Found 672 images belonging to 64 classes.
Found 224 images belonging to 64 classes.


In [15]:
#Load the pretrained Network
nasnet_model = NASNetLarge( 
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(img_height,img_width,3),
    pooling=None,
    classes=1000
    
)
print("pretrained Network is loaded")

343613440/343610240 [==============================] - 27s 0us/step
pretrained Network is loaded


In [16]:
nasnet_model.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 331, 331, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 165, 165, 96) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

In [17]:
# Freeze the layers
for layer in nasnet_model.layers:
    layer.trainable = False
print("Pretrained layers are freezed")

Pretrained layers are freezed


In [18]:
# Model Creation
model = Sequential()
model.add(nasnet_model)
#add fully connected layer:
input_layer=model.add(Flatten())
hidden_layer=model.add(Dense(128, activation='relu', name='hidden_layer'))
classification_layer=model.add(Dense(64, activation='relu', name='classification_layer'))
output_layer=model.add(Dense(64, activation='softmax', name='output_layer')) 
print("All layers top of pretrained layers are developed")

All layers top of pretrained layers are developed


In [19]:
# Model compilation
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [20]:
# model weights saving after each epoch
checkpoint = ModelCheckpoint(filepath = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
print('Check point is created')

Check point is created


In [21]:
history = model.fit_generator( train_generator, callbacks = callbacks_list, nb_epoch=nb_epochs, validation_data=validation_generator)
print('Training Completed!')

Epoch 1/50
21/21 [==============================] - 538s 26s/step - loss: 13.6917 - accuracy: 0.0714 - val_loss: 5.9023 - val_accuracy: 0.0625
Epoch 2/50
21/21 [==============================] - 34s 2s/step - loss: 4.2188 - accuracy: 0.2158 - val_loss: 5.3888 - val_accuracy: 0.0714
Epoch 3/50
21/21 [==============================] - 35s 2s/step - loss: 2.8890 - accuracy: 0.3244 - val_loss: 4.9266 - val_accuracy: 0.0536
Epoch 4/50
21/21 [==============================] - 34s 2s/step - loss: 2.4638 - accuracy: 0.4092 - val_loss: 5.7696 - val_accuracy: 0.0536
Epoch 5/50
21/21 [==============================] - 35s 2s/step - loss: 2.0657 - accuracy: 0.4821 - val_loss: 6.6042 - val_accuracy: 0.0536
Epoch 6/50
21/21 [==============================] - 35s 2s/step - loss: 1.7470 - accuracy: 0.5759 - val_loss: 4.9372 - val_accuracy: 0.0938
Epoch 7/50
21/21 [==============================] - 35s 2s/step - loss: 1.6224 - accuracy: 0.6042 - val_loss: 6.2382 - val_accuracy: 0.0714
Epoch 8/50
21/21 

In [22]:
# save model and architecture to single file
model.save('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_nasnet_model.h5')
model.summary()

print("Saved model to disk")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
flatten_1 (Flatten)          (None, 487872)            0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               62447744  
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 64)                4160      
Total params: 147,376,978
Trainable params: 62,460,160
Non-trainable params: 84,916,818
_________________________________________________________________
Saved model to disk


In [23]:
# Loading saved model from Drive.
from keras.models import load_model
model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_nasnet_model.h5')
print("Model is Loaded")
model.summary()

Model is Loaded
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
flatten_1 (Flatten)          (None, 487872)            0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               62447744  
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 64)                4160      
Total params: 147,376,978
Trainable params: 62,460,160
Non-trainable params: 84,916,818
_________________________________________________________________


In [0]:
import os
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train/'
def get_images(fish):
    """Load files from train folder"""
    fish_dir = train_data_dir+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images

In [25]:
pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 106kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [0]:
def read_image(src):
    import os
    from scipy import misc
    filepath=src
    im=misc.imread(filepath)
    import scipy.misc  as mc
     
    return mc.imresize(im,(img_width, img_height))

In [27]:
files = []
y_all = []
for fish in SIGNATURE_CLASSES:
    fish_files = get_images(fish)
    files.extend(fish_files)
    
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)
print(len(files))
print(len(y_all))


19 photos of 001
19 photos of 002
18 photos of 003
18 photos of 004
18 photos of 006
18 photos of 009
18 photos of 012
9 photos of 013
18 photos of 014
18 photos of 015
18 photos of 016
9 photos of 017
9 photos of 018
9 photos of 019
9 photos of 020
9 photos of 021
9 photos of 022
9 photos of 023
9 photos of 024
9 photos of 025
9 photos of 026
9 photos of 027
9 photos of 028
9 photos of 029
9 photos of 030
9 photos of 031
9 photos of 032
9 photos of 033
9 photos of 034
9 photos of 035
9 photos of 036
9 photos of 037
9 photos of 038
9 photos of 039
9 photos of 040
9 photos of 041
9 photos of 042
9 photos of 043
9 photos of 044
9 photos of 045
9 photos of 046
9 photos of 047
6 photos of 048
9 photos of 049
9 photos of 050
9 photos of 051
9 photos of 052
9 photos of 053
9 photos of 054
9 photos of 055
9 photos of 056
9 photos of 057
9 photos of 058
9 photos of 059
9 photos of 060
9 photos of 061
9 photos of 062
9 photos of 063
9 photos of 064
9 photos of 065
9 photos of 066
12 photos of 0

In [28]:
X_all = np.ndarray((len(files),img_width, img_height , 3), dtype=np.uint8)
print(X_all.shape)
for i, im in enumerate(files): 
    X_all[i] = read_image(train_data_dir+im)
    print(X_all[i])   
   
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))


print(X_all.shape)

Streaming output truncated to the last 5000 lines.
[[[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 ...

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]]
[[[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 ...

 

In [29]:
import os
test_files = [im for im in os.listdir(TEST_DIR)]
test = np.ndarray((len(test_files), img_width, img_height, 3), dtype=np.uint8)

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)
    
test_preds = model.predict(test, verbose=1)
submission = pd.DataFrame(test_preds, columns=SIGNATURE_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()

submission.to_csv('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureResultsnasnet.csv',index=False)

12/12 [==============================] - 8s 690ms/step


In [31]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np
from keras.models import load_model
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Click to know Name " )
output = widgets.Output()

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_nasnet_model.h5')
img_path = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/11_052.png'
img = image.load_img(img_path, target_size=(331, 331))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)
print(block4_pool_features)
label_index=block4_pool_features.argmax()
print(label_index)
def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    print("Signature is of  "  +SIGNATURE_CLASSES[label_index])

button.on_click(on_button_clicked)
display(button, output)

[[2.46259827e-13 3.12371999e-18 3.31624520e-11 1.80972029e-06
  8.03524192e-11 1.83515980e-11 4.32912373e-07 2.27875702e-07
  6.50418044e-12 5.97607291e-14 6.13580853e-11 1.24452764e-17
  5.42877939e-12 1.38210943e-09 1.90656357e-09 7.56932952e-15
  3.21606931e-04 3.46818546e-10 1.22426997e-07 2.74355415e-07
  2.37329288e-11 7.61444305e-17 2.17480238e-06 2.25384824e-07
  5.83667017e-04 2.35148793e-12 5.63869359e-14 1.44455798e-05
  1.22915983e-11 9.90885650e-13 8.18110639e-05 2.14026950e-05
  1.49932900e-03 1.00354191e-04 1.71700487e-09 9.31759600e-11
  9.84228046e-11 3.84341689e-07 3.66455495e-01 5.27967131e-07
  6.36697401e-13 4.26811795e-08 1.68175127e-08 2.06730032e-11
  9.02787690e-07 3.69124718e-05 3.67782637e-15 7.59461409e-06
  4.12995471e-10 5.46353638e-01 4.59728199e-06 6.92179981e-07
  5.63563715e-15 1.94740277e-02 5.81572840e-06 7.65209300e-17
  5.21145090e-02 1.63333512e-13 2.95015576e-04 2.28735142e-10
  6.68253257e-12 1.26219401e-02 1.47981437e-17 1.59619293e-13]]
49


Button(description='Click to know Name ', style=ButtonStyle())

Output()

In [32]:
from keras.models import load_model

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_nasnet_model.h5')
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(validation_generator)
print('test loss, test acc:', results)


# Evaluate on test data
7/7 [==============================] - 17s 2s/step
test loss, test acc: [15.079809188842773, 0.15625]


In [33]:
# Extracting Features from classification Layer
from keras.models import Model
layer_name= 'classification_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
print("Imtermediate model is constructed")


Imtermediate model is constructed


In [34]:
# Compilation of intermediate model
intermediate_layer_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [36]:
# Saving intermediate model
intermediate_layer_model.save('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_nasnettfolder_Intermediatemodel.h5')
intermediate_layer_model.summary()

print("Saved Intermediate model to disk")

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet_input (InputLayer)    (None, 331, 331, 3)       0         
_________________________________________________________________
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
flatten_1 (Flatten)          (None, 487872)            0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               62447744  
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 147,372,818
Trainable params: 62,456,000
Non-trainable params: 84,916,818
_________________________________________________________________
Saved Intermediate model to disk


In [37]:
# Loading Intermediate Model
from keras.models import load_model

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_nasnettfolder_Intermediatemodel.h5')
print("Intermediate model is loaded")

Intermediate model is loaded


In [38]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet_input (InputLayer)    (None, 331, 331, 3)       0         
_________________________________________________________________
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
flatten_1 (Flatten)          (None, 487872)            0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               62447744  
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 147,372,818
Trainable params: 62,456,000
Non-trainable params: 84,916,818
_________________________________________________________________


In [39]:
# Training Label feature identification(y_train)
import numpy as np
batch_size=32
sample_count=672
features = np.zeros(shape=(672, 64))  # Must be equal to the output of the convolutional base
labels = np.zeros(shape=(672, 64))
i = 0
for inputs_batch, labels_batch in train_generator:
  features_batch = model.predict(inputs_batch)
  features[i * batch_size: (i + 1) * batch_size] = features_batch
  labels[i * batch_size: (i + 1) * batch_size] = labels_batch
  i += 1
  if i*batch_size  >= sample_count:
    break
print(labels.shape)
print(features.shape)


(672, 64)
(672, 64)


In [40]:
 print(labels)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [41]:
#identification of training Labels
features_train=features
print(features_train.shape)
labels_train=labels
print(labels_train.shape)
print(labels)
#print(features_train)


(672, 64)
(672, 64)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [42]:
# identification of test labels
import numpy as np
batch_size=32
sample_count=224
features_test = np.zeros(shape=(224, 64))  # Must be equal to the output of the convolutional base
labels_test = np.zeros(shape=(224,64))
i = 0
for inputs_batch, labels_batch in validation_generator:
  features_batch = model.predict(inputs_batch)
  features_test[i * batch_size: (i + 1) * batch_size] = features_batch
  labels_test[i * batch_size: (i + 1) * batch_size] = labels_batch
  i += 1
  if i*batch_size  >= sample_count:
    break
print(labels_test.shape)
print(features_test.shape)
print(labels_test)

(224, 64)
(224, 64)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [43]:
# #identification of testing Labels
print(features_test.shape)
#labels_test=np.expand_dims(labels_test, axis=1)
print(labels_test.shape)
print(features_test)

(224, 64)
(224, 64)
[[ 6.94560862  3.78835797  0.         ...  0.         25.62225914
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.         10.41393566
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [44]:
print(labels_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [45]:
# SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
print(X_train.shape)
print(labels_train.shape)
X_test = sc.transform(features_test)
from sklearn.utils import check_X_y
X, y = check_X_y(
            X_train, labels_train, multi_output=True)
# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X,y)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='binary')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='binary')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='binary')
print('F-Measure: %.3f' % (score*100))

(672, 64)
(672, 64)


ValueError: ignored

In [46]:
# Kernel SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='binary')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='binary')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='binary')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored

In [47]:
# Random Forest
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 11.161
Precision: 83.333
Recall: 11.161
F-Measure: 19.685


In [48]:
# Decision Tree
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 34.821
Precision: 34.821
Recall: 34.821
F-Measure: 34.821


In [49]:
# Naive Bayes
# Feature Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_X_y
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)
X, y = check_X_y(X_train, labels_train, accept_sparse=False, accept_large_sparse=True, dtype='numeric', order=None, copy=False, force_all_finite=True, ensure_2d=True, allow_nd=False, multi_output=True, ensure_min_samples=1, ensure_min_features=1, y_numeric=True, estimator=None)
# Fitting KernelSVM to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X, y)
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored

In [50]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier =  KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 29.911
Precision: 73.626
Recall: 29.911
F-Measure: 42.540


In [51]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)
from sklearn.utils import check_X_y
X, y = check_X_y(
            X_train, labels_train, multi_output=True)
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, y)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored